In [ ]:
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Conv2D,Conv2DTranspose,Activation
from tensorflow.keras.layers import Concatenate,Dense,MaxPooling2D,UpSampling2D,Input
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras import backend as K

import tensorflow.keras as keras
import tensorflow.compat.v1 as tf
import numpy as np
import warnings
import os
import glob

warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
tf.config.optimizer.set_jit(True)

In [ ]:
home_path = '/data1/home/weisting/CycleGan/Experiment5/Fill_Input_Version/Target_Embedding/Fill_TE_Data/'
Air_train = np.load(home_path+'AIR_Align_Kriging_Training_del.npy', mmap_mode='r')
Air_test =  np.load(home_path+'AIR_Align_Kriging_Testing_del.npy', mmap_mode='r')

Sat_train = np.load(home_path+'SAT_Align_fill_Training_del.npy', mmap_mode='r')
Sat_test = np.load(home_path+'SAT_Align_fill_Testing_del.npy', mmap_mode='r')

train = np.concatenate([Air_train,Sat_train])
test = np.concatenate([Air_test,Sat_test])

print(train.shape, test.shape)

In [ ]:
def Encoder(channel_number):
    input_img = Input(shape=(348,204,1))
    e1 = Conv2D(16, (3, 3), activation='tanh', padding='same')(input_img)
    e2 = MaxPooling2D((2, 2), padding='same')(e1)
    e3 = Conv2D(8, (3, 3), activation='tanh', padding='same')(e2)
    e4 = MaxPooling2D((2, 2), padding='same')(e3)
    e5 = Conv2D(channel_number, (3, 3), activation='tanh', padding='same')(e4)
    e6 = MaxPooling2D((2, 2), padding='same')(e5)
    e7 = Activation('tanh')(e6)
    return Model(input_img, e7)

def Decoder(channel_number):
    input_img = Input(shape=(44,26,channel_number))
    d1 = Conv2D(8, (3, 3), activation='relu', padding='same')(input_img)
    d2 = UpSampling2D((2, 2))(d1)
    d3 = Conv2D(16, (3, 3), activation='relu', padding='same')(d2)
    d4 = UpSampling2D((2, 2))(d3)
    d5 = Conv2D(32, (3, 3), activation='relu')(d4)
    d6 = UpSampling2D((2, 2))(d5)
    d7 = Conv2D(1, (3, 3), activation='relu', padding='same')(d6)
    return Model(input_img, d7)

In [ ]:
channel_number = 6
path = 'AE_result'
if not os.path.exists(path):os.makedirs(path)
K.clear_session()
x = Input(shape=(348, 204, 1))
opt = keras.optimizers.Adam(learning_rate=0.0005)

Encoder_model = Encoder(channel_number)
Decoder_model = Decoder(channel_number)

Autoencoder = Model(x,Decoder_model(Encoder_model(x)))
Autoencoder.compile(loss='mse',optimizer=opt)

filepath=path+'/{epoch:04d}-train_{loss:f}-test_{val_loss:f}.h5'
earlystopper = EarlyStopping(monitor='val_loss', patience=50, verbose=1, min_delta=0.05)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.2, patience=15, min_lr = 2e-7, verbose = 1)                   
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='auto')
callbacks_list = [earlystopper, reduce_lr, checkpoint]

history = Autoencoder.fit(train,train,
                          validation_data=(test,test),
                          batch_size=32, epochs=2000,verbose=1,callbacks=callbacks_list)

del Encoder_model
del Decoder_model
del Autoencoder

In [ ]:
def glob_all(dir_path):
    file_list = glob.glob(os.path.join(dir_path,'*.h5'))
    inside = os.listdir(dir_path)
    for dir_name in inside:
        if os.path.isdir(os.path.join(dir_path,dir_name)):
            file_list.extend(glob_all(os.path.join(dir_path,dir_name)))
    return file_list

def MSE(y, y_pred):
    return np.round(np.mean((y - y_pred)**2),4)

Encoder_model = Encoder(channel_number)
Decoder_model = Decoder(channel_number)
Autoencoder = Model(x,Decoder_model(Encoder_model(x)))

Autoencoder.load_weights(glob_all('AE_result')[0])
np.save('Unpair_train_Code6.npy',Encoder_model.predict(train,batch_size=4,verbose=1))